In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
try:
  #Get whisper
  !pip install -U openai-whisper
  #Get the testing data
  !wget https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz
except Exception as e:
  print(f"Couldn't get because of error : {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 3.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=cbbd1c2553e24cd52722512af6c84f9ce7606d2222f0c2752fd918179de3654a
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
--2024-04-26 11:00:41--  https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz
Resolving asr.iitm.ac.in (asr.iitm.ac.in)... 103.158.43.117
Connecting to asr.iitm.ac.in (asr.iitm.ac.in)|103.158.43.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62063203 (59M) [appl

In [3]:
#Unzip the tar.gz file in contents
!tar -xf GV_Eval_3h.tar.gz

In [4]:
# Loading whisper
import whisper
try:
  variant = input('Enter variant name : ')
  variant = variant.lower()

  model = whisper.load_model(variant)
except Exception as e:
  print(f"Cannot import the {variant} variant for whisper due to : {e}")

Enter variant name :  large


100%|██████████████████████████████████████| 2.88G/2.88G [00:17<00:00, 174MiB/s]


In [6]:
!pip install natsort

In [8]:
# Transcribing all the audio and saving results in a file
import glob
import os
from natsort import natsorted

# Path to the parent dir
path_to_audio = '/kaggle/working/GV_Eval_3h/Audio'

# Setting a pattern through glob to find all the audio files
audio_files_names = glob.glob(path_to_audio + '**/' + '*.mp3', recursive=True)

# Sorting the file paths based on the file names
audio_files_names = natsorted(audio_files_names, key=lambda x: os.path.basename(x))

print(audio_files_names)

['/kaggle/working/GV_Eval_3h/Audio/01-00004-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00008-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00031-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00071-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00078-01.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00093-01.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00097-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00119-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00121-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00129-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00131-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00142-01.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00144-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00159-01.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00159-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00173-03.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00262-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00264-02.mp3', '/kaggle/working/GV_Eval_3h/Audio/01-00281-01.mp3', '/kaggle/wo

In [9]:
l = len(audio_files_names)
print(f"There are {l} files in the audio directory")

There are 1032 files in the audio directory


In [13]:
# Opening a file and writing all the results of transcription in it
with open('/kaggle/working/results.txt', 'w') as f:
  for audio_file in audio_files_names:
    predict = model.transcribe(audio_file, language='hi')
    audio_name = audio_file.split('/')[-1].split('.')[0]
    f.write(audio_name + " " + predict['text']+'\n')

In [14]:
#installing jiwer for wer calculation
try:
    !pip install jiwer
    from jiwer import wer
except Exception as e:
    print(f"Couldn't import due to error {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.6 MB/s eta 0:00:00a 0:00:01


In [15]:
# Extracting all the ground truths from the ground truths file 
def extract_all_lines(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(line)

In [16]:
extract_all_lines('/kaggle/working/GV_Eval_3h/text', '/kaggle/working/ground_truths.txt')

In [21]:
%%writefile eval_wer.py
import jiwer
import argparse

def calculate_wer(ground_truth_file, predictions_file):
    # Read the ground truth and predictions
    with open(ground_truth_file, 'r', encoding='utf-8') as f:
        ground_truth = f.readlines()

    with open(predictions_file, 'r', encoding='utf-8') as f:
        predictions = f.readlines()

    # Check if the number of lines in both files are equal
    if len(ground_truth) != len(predictions):
        raise ValueError("The number of lines in ground truth and predictions files are not equal.")

    # Extract the transcripts from the files
    ground_truth_transcripts = [line.strip() for line in ground_truth]
    predicted_transcripts = [line.strip() for line in predictions]

    # Calculate the WER
    wer = jiwer.wer(ground_truth_transcripts, predicted_transcripts)

    print(f'The WER is {wer} for the ground_truth_transcripts against predicted_transcripts')

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Calculate Word Error Rate (WER)")
    parser.add_argument("--ground_truth_file", type=str, help="Path to the ground truth file")
    parser.add_argument("--predictions_file", type=str, help="Path to the predictions file")

    args = parser.parse_args()

    wer = calculate_wer(args.ground_truth_file, args.predictions_file)

Overwriting eval_wer.py


In [23]:
!python /kaggle/working/eval_wer.py \
--ground_truth_file=/kaggle/working/ground_truths.txt \
--predictions_file=/kaggle/working/results.txt

The WER is 0.5017995225029398 for the ground_truth_transcripts against predicted_transcripts
